In [1]:
import sys

sys.path.append('../')

import time
from canopus import *
from canopus.utils import *
from qiskit.transpiler import CouplingMap, PassManager
from pytket.utils import compare_unitaries
from qiskit import qasm2

from qiskit.circuit.library import QFT

# configure logging
# logging.basicConfig(
#     level=logging.INFO,
#     format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
#     handlers=[logging.StreamHandler()]
# )


qc = qasm2.loads("""
OPENQASM 2.0;
include "qelib1.inc";
qreg q[6];
h q[0];
h q[2];
h q[5];
z q[0];
cx q[1],q[2];
cx q[4],q[5];
cx q[0],q[1];
cx q[2],q[3];
h q[2];
h q[3];
cx q[1],q[2];
cx q[3],q[5];
z q[3];
cx q[3],q[4];
cx q[0],q[3];
""")

circ = qiskit_to_tket(qc)
circ = rebase_to_tk2(circ)
qc = tket_to_qiskit(circ)

console.rule('Original circuit')

# print(qc)
assert compare_unitaries(circ.get_unitary(), qc2mat(qc))

print(qiskit_to_tket(qc).get_commands())

coupling_map = CouplingMap.from_line(num_qubits=qc.num_qubits)
# coupling_map = CouplingMap.from_grid(np.ceil(np.sqrt(qc.num_qubits)).astype(int), np.ceil(np.sqrt(qc.num_qubits)).astype(int))
backend = CanopusBackend(coupling_map, 'cx', 'xy')

console.print('Pulse duration: {:.4f}'.format(backend.cost_estimator.eval_circuit_duration(qc)))

console.rule('SABRE mapping')
start = time.perf_counter()
pm = PassManager(SabreMapping(backend, seed=123))
qc_sabre = pm.run(qc)
end = time.perf_counter()
# print(qc_sabre)
qc_sabre_opt = tket_to_qiskit(rebase_to_tk2(qiskit_to_tket(qc_sabre)))
# print(qc_sabre_opt)
console.print('Pulse duration: {:.4f}'.format(backend.cost_estimator.eval_circuit_duration(qc_sabre_opt)))
console.print('Time taken for Canopus mapping: {:.4f} seconds'.format(end - start))

console.rule('Canopus mapping')
start = time.perf_counter()
pm = PassManager(CanopusMapping(backend, seed=123))
qc_canopus = pm.run(qc)
end = time.perf_counter()
# print(qc_canopus)
qc_canopus_opt = tket_to_qiskit(rebase_to_tk2(qiskit_to_tket(qc_canopus)))
# print(qc_canopus_opt)
console.print('Pulse duration: {:.4f}'.format(backend.cost_estimator.eval_circuit_duration(qc_canopus_opt)))
console.print('Time taken for Canopus mapping: {:.4f} seconds'.format(end - start))


──────────────────────────────────────────────── Original circuit ─────────────────────────────────────────────────

[U3(2.99557, 1.14159, 1.5708) q[1];, U3(0.712389, 0, 1.5708) q[2];, U3(0.712389, 0.429204, 0.712389) q[3];, U3(2.99557, 1.14159, 1.5708) q[4];, U3(0.712389, 0, 1.14159) q[5];, TK2(0.5, 0, 0) q[1], q[2];, TK2(0.5, 0, 0) q[4], q[5];, U3(0.712389, 0, 1.5708) q[1];, U3(1.5708, 0, 1.5708) q[2];, U3(0.712389, 0, 1.5708) q[4];, TK2(0.5, 0, 0) q[0], q[1];, TK2(0.5, 0, 0) q[2], q[3];, U3(1.5708, 0, 1.5708) q[1];, U3(0.712389, 0.429204, 0.712389) q[2];, U3(0, 0.429204, 0.712389) q[3];, TK2(0.5, 0, 0) q[1], q[2];, TK2(0.5, 0, 0) q[3], q[5];, U3(1.5708, 0, 1.14159) q[1];, U3(0, 0.429204, 0.712389) q[2];, TK2(0.5, 0, 0) q[3], q[4];, U3(0, 0.429204, 0.712389) q[5];, U3(0.712389, 0, 1.5708) q[3];, U3(0, 0.429204, 0.712389) q[4];, TK2(0.5, 0, 0) q[0], q[3];, U3(1.5708, 0, 1.14159) q[0];, U3(0, 0.429204, 0.712389) q[3];]


Pulse duration: 5.0000

────────────────────────────────────────────────── SABRE mapping ──────────────────────────────────────────────────

Pulse duration: 11.0000

Time taken for Canopus mapping: 0.0687 seconds

───────────────────────────────────────────────── Canopus mapping ─────────────────────────────────────────────────

Pulse duration: 10.0000

Time taken for Canopus mapping: 0.0975 seconds

In [2]:
from pytket import passes
from pytket.circuit.display import render_circuit_jupyter

In [3]:
c = qiskit_to_tket(qc_canopus)
gate_counts(c)
render_circuit_jupyter(c)

In [ ]:
passes.SynthesiseTK().apply(c)
render_circuit_jupyter(c)

In [5]:
rebase_to_zzphase(c)

[TK1(0.143806, 0.566371, 1.0708) q[0]; TK1(0.380519, 3.62245, 0.869723) q[1]; TK1(0.0707963, 1.49779, 0.212389) q[2]; TK1(0.380519, 3.62245, 1.29893) q[3]; TK1(0.143806, 0.566371, 1.0708) q[4]; TK1(0.143806, 1.5708, 0.358407) q[5]; ZZPhase(0.333333) q[0], q[1]; ZZPhase(0.333333) q[4], q[3]; TK1(0, 0, 0.429204) q[0]; TK1(0.929204, 3.5708, 3.0708) q[1]; TK1(0.929204, 3.5708, 3.0708) q[3]; TK1(0, 0, 0.429204) q[4]; ZZPhase(0.166667) q[0], q[1]; ZZPhase(0.166667) q[4], q[3]; TK1(0.108036, 3.27609, 0.888986) q[0]; TK1(0.942382, 3.55306, 1.15861) q[1]; TK1(3.17762, 3.68219, 0.223452) q[3]; TK1(0.108036, 3.27609, 0.888986) q[4]; ZZPhase(0.333333) q[3], q[2]; ZZPhase(0.333333) q[5], q[4]; TK1(0.929204, 3.5708, 3.0708) q[2]; TK1(0, 0, 0.429204) q[3]; TK1(0.929204, 3.5708, 3.0708) q[4]; TK1(0, 0, 0.429204) q[5]; ZZPhase(0.166667) q[3], q[2]; ZZPhase(0.166667) q[5], q[4]; TK1(0.513102, 3.59361, 0.535963) q[2]; TK1(0.479963, 3.70564, 1.70123) q[3]; TK1(3.17762, 3.68219, 0.223452) q[4]; TK1(1.52613

In [ ]:
from qiskit.circuit.library import RZXGate

In [6]:
rebase_to_zzphase(qc_canopus).draw()

/Users/anan/git-projects/quantum/canopus/experiments/../canopus/utils.py:59: UserWarning: !!!!!! Unsupported pytket circuit type: {OpType.TK1, OpType.ZZPhase}
  warnings.warn('!!!!!! Unsupported pytket circuit type: {}'.format(set(gate_counts(circ).keys())))


┌──────────────────────────┐             ┌───────────────────┐    »
q_0: ┤ U3(8.0625,-1.119,4.9348) ├─■───────────┤ U3(0,-π/2,2.9192) ├────»
     ├─────────────────────────┬┘ │ZZ(π/3) ┌──┴───────────────────┴───┐»
q_1: ┤ U3(11.38,5.9078,4.3031) ├──■────────┤ U3(11.218,1.3484,11.218) ├»
     ├─────────────────────────┴┐          └──────────────────────────┘»
q_2: ┤ U3(4.7054,-1.3484,2.238) ├──────────────────────────────────────»
     ├─────────────────────────┬┘          ┌──────────────────────────┐»
q_3: ┤ U3(11.38,5.9078,5.6515) ├──■────────┤ U3(11.218,1.3484,11.218) ├»
     ├─────────────────────────┴┐ │ZZ(π/3) └──┬───────────────────┬───┘»
q_4: ┤ U3(8.0625,-1.119,4.9348) ├─■───────────┤ U3(0,-π/2,2.9192) ├────»
     ├──────────────────────────┤             └───────────────────┘    »
q_5: ┤ U3(4.9348,-1.119,2.6968) ├──────────────────────────────────────»
     └──────────────────────────┘                                      »
«               ┌───────────────────────────┐          »
«q_0: ─■────────┤ U3(10.292,-1.2314,4.3636) ├──────────»
«      │ZZ(π/6) └┬──────────────────────────┤          »
«q_1: ─■─────────┤ U3(11.162,1.3898,5.2107) ├──────────»
«                └──────────────────────────┘          »
«q_2: ────────────────────────────────────────■────────»
«                ┌─────────────────────────┐  │ZZ(π/3) »
«q_3: ─■─────────┤ U3(11.568,8.412,2.2728) ├──■────────»
«      │ZZ(π/6) ┌┴─────────────────────────┴┐          »
«q_4: ─■────────┤ U3(10.292,-1.2314,4.3636) ├─■────────»
«               └───────────────────────────┘ │ZZ(π/3) »
«q_5: ────────────────────────────────────────■────────»
«                                                      »
«                                                                          »
«q_0: ─────────────────────────────────────────────────────────────────────»
«                                                                          »
«q_1: ─────────────────────────────────────────────────────────────────────»
«     ┌──────────────────────────┐           ┌───────────────────────────┐ »
«q_2: ┤ U3(11.218,1.3484,11.218) ├─■─────────┤ U3(11.29,0.041162,3.2546) ├─»
«     └──┬───────────────────┬───┘ │ZZ(π/6) ┌┴───────────────────────────┴┐»
«q_3: ───┤ U3(0,-π/2,2.9192) ├─────■────────┤ U3(11.642,-0.062949,6.9154) ├»
«     ┌──┴───────────────────┴───┐          └─┬─────────────────────────┬─┘»
«q_4: ┤ U3(11.218,1.3484,11.218) ├─■──────────┤ U3(11.568,8.412,2.2728) ├──»
«     └──┬───────────────────┬───┘ │ZZ(π/6)   ├─────────────────────────┴┐ »
«q_5: ───┤ U3(0,-π/2,2.9192) ├─────■──────────┤ U3(10.921,3.2237,2.5629) ├─»
«        └───────────────────┘                └──────────────────────────┘ »
«                                                     »
«q_0: ────────────────────────────────────────────────»
«               ┌──────────────────────────┐          »
«q_1: ─■────────┤ U3(11.218,1.3484,11.218) ├─■────────»
«      │ZZ(π/3) └──┬───────────────────┬───┘ │ZZ(π/6) »
«q_2: ─■───────────┤ U3(0,-π/2,2.9192) ├─────■────────»
«               ┌──┴───────────────────┴───┐          »
«q_3: ─■────────┤ U3(11.218,1.3484,11.218) ├─■────────»
«      │ZZ(π/3) └──┬───────────────────┬───┘ │ZZ(π/6) »
«q_4: ─■───────────┤ U3(0,-π/2,2.9192) ├─────■────────»
«                  └───────────────────┘              »
«q_5: ────────────────────────────────────────────────»
«                                                     »
«                                                                       »
«q_0: ──────────────────────────────────────────────────────────────────»
«     ┌──────────────────────────┐             ┌───────────────────┐    »
«q_1: ┤ U3(10.532,10.416,13.331) ├─■───────────┤ U3(0,-π/2,6.5056) ├────»
«     ├──────────────────────────┤ │ZZ(π/6) ┌──┴───────────────────┴───┐»
«q_2: ┤ U3(10.921,3.2237,2.5629) ├─■────────┤ U3(4.9348,1.3484,4.9348) ├»
«     ├──────────────────────────┤          └──────────────────────────┘»
«q_3: ┤ U3(10.532,10.416,13.331) ├──────────────────────────────────